In [190]:
import json
import pandas as pd
from gurobipy import Model, GRB, quicksum, GurobiError # Added GurobiError for IIS check later
from collections import deque
import math # Needed for ceiling if using /N method, although sticking to N*A

# CIV 6 DISTRICT PLACEMENT OPTIMIZATION MODEL
# INPUT SETS / DATA INITALIZATION

# parse input data from civ_map_data.json into a pandas dataframe
# Make sure 'civ_map_data.json' is in the same directory or provide the correct path
try:
    with open('civ_map_data.json') as f:
        data = json.load(f)
    df = pd.DataFrame(data['tiles'])
    print("Loaded civ_map_data.json successfully.")
    # print(df.head()) # Optional: print head to verify
except FileNotFoundError:
    print("ERROR: civ_map_data.json not found. Please ensure it's in the correct directory.")
    # exit() # Or handle appropriately
except Exception as e:
    print(f"ERROR: Could not load or parse civ_map_data.json: {e}")
    # exit() # Or handle appropriately


# assume that the city center is selected at the tile with the highest normalized score.
# Ensure the dataframe 'df' and the column 'normalized_score' exist
if 'normalized_score' in df.columns:
    best_tile = df.loc[df['normalized_score'].idxmax()]


    # cc_x = int(best_tile['x']) # Ensure integer coords
    # cc_y = int(best_tile['y'])

    cc_x = 52
    cc_y = 18

    city_center_coords = (cc_x, cc_y)
    print(f"City Center coordinates selected: {city_center_coords}")
else:
    print("ERROR: 'normalized_score' column not found in DataFrame. Cannot determine City Center.")
    # Handle error - e.g., set a default CC or exit
    # city_center_coords = (some_default_x, some_default_y)
    # exit()

Loaded civ_map_data.json successfully.
City Center coordinates selected: (52, 18)


In [191]:
# --- Constants and Data Structures ---

# Set of Districts to consider
ALL_DISTRICTS = [
    "Campus", "Holy Site", "Harbor", "Government Plaza", # Ensured GP is present
    "Theater Square", "Entertainment Complex", "Commercial Hub",
    "Industrial Zone", "Aqueduct", "Water Park", "Dam", "Canal"
    # Add others if needed, e.g., Diplomatic Quarter, Preserve
]

# Unique Districts (usually matches ALL_DISTRICTS except maybe Neighborhood)
DISTRICTS_UNIQUE = [
     "Campus", "Holy Site", "Harbor", "Government Plaza", # Ensured GP is present
     "Theater Square", "Entertainment Complex", "Commercial Hub",
     "Industrial Zone", "Aqueduct", "Water Park", "Dam", "Canal"
     # Add others if needed
]

# Mutually Exclusive Sets
DISTRICTS_MUTUALLY_EXCLUSIVE = [
    {"Entertainment Complex", "Water Park"}
]

DISTRICT_BASE_VALUE = {
    "Campus": 2.0,
    "Industrial Zone": 2.0,
    "Commercial Hub": 2.0,
    "Theater Square": 2.0,
    "Holy Site": 2.0,
    "Harbor": 2.0,
    "Government Plaza": 3.0 # Higher value since it provides adjacency to others
    # Add values for other districts
}

# Features relevant for adjacency or placement
FEATURES = {
    "Mountain": ["TERRAIN_GRASS_MOUNTAIN", "TERRAIN_DESERT_MOUNTAIN", "TERRAIN_PLAINS_MOUNTAIN", "TERRAIN_TUNDRA_MOUNTAIN", "TERRAIN_SNOW_MOUNTAIN"],
    "Coast": ["TERRAIN_COAST"],
    "River": ["River"], # Placeholder for river logic
    "Lake": ["TERRAIN_LAKE"],
    "Oasis": ["TERRAIN_DESERT_OASIS"],
    "Geothermal Fissure": ["FEATURE_GEOTHERMAL_FISSURE"],
    "Reef": ["FEATURE_REEF"],
    "Woods": ["FEATURE_WOODS"],
    "Rainforest": ["FEATURE_JUNGLE"],
    "Marsh": ["FEATURE_MARSH"],
    "Floodplains": ["FEATURE_FLOODPLAINS", "FEATURE_FLOODPLAINS_GRASSLAND", "FEATURE_FLOODPLAINS_PLAINS"], # Added Floodplains
    "NaturalWonder": ["NaturalWonder"], # Placeholder if you add specific NW logic
    "Wonder": ["Wonder"], # Placeholder for Built Wonders if treated as features
    "Strategic": ["StrategicResource"], # Placeholder
    "Luxury": ["LuxuryResource"] # Placeholder
}

# Terrain Adjacency Bonuses
TERRAIN_BONUS_VALUE = {
    ("Campus", "Mountain"): 1, ("Campus", "Reef"): 2, ("Campus", "Geothermal Fissure"): 2,
    ("Campus", "Rainforest"): 0.5,
    ("Holy Site", "Mountain"): 1, ("Holy Site", "Woods"): 0.5,
    ("Holy Site", "Natural Wonder"): 2, # Assumes "NaturalWonder" feature exists
    ("Commercial Hub", "River"): 2,     # Handled by river logic later
    ("Theater Square", "Wonder"): 2,      # Assumes "Wonder" feature exists for built wonders
    ("Industrial Zone", "Strategic"): 1, # Example if needed
    ("Aqueduct", "Geothermal Fissure"): 2, # Example bonus TO aqueduct
    # Note: River bonus for CH is handled separately based on river edge data
}
# Ensure "River" key exists if used for CH bonus calculation later
RIVER_FEATURE_KEY = "River"

# City Center Adjacency Bonuses
CC_BONUS_VALUE = { "Harbor": 2 } # Default others to 0 later

# District-to-District Adjacency Bonuses
# Format: (District Receiving Bonus, Adjacent District/Feature Providing Bonus): Bonus Value
DISTRICT_PAIR_BONUS = {
    # --- Original Pairs ---
    ('Commercial Hub', 'Harbor'): 2,
    
    # ('Theater Square', 'Wonder'): 2, # Covered by TERRAIN_BONUS_VALUE if Wonder is a Feature
    ('Theater Square', 'Entertainment Complex'): 1,
    ('Theater Square', 'Water Park'): 1,
    ('Industrial Zone', 'Aqueduct'): 2,
    ('Industrial Zone', 'Dam'): 2,
    ('Industrial Zone', 'Canal'): 2,
    # --- Government Plaza Provided Bonuses ---
    ('Campus',          'Government Plaza'): 1, # <--- Set back to 1 (or 10 for testing)
    ('Holy Site',       'Government Plaza'): 1,
    ('Harbor',          'Government Plaza'): 1,
    ('Commercial Hub',  'Government Plaza'): 1,
    ('Industrial Zone', 'Government Plaza'): 1,
    ('Theater Square',  'Government Plaza'): 1,
}
# Ensure consistent district names/keys! Check spelling carefully.

# Default CC Bonus to 0 for districts not listed
for dist in ALL_DISTRICTS: CC_BONUS_VALUE.setdefault(dist, 0)

# Other Constants
RADIUS = 3
MAX_ADJACENT_TILES = 6 # Max neighbors in hex grid

IMPASSABLE_TERRAIN = {
    "TERRAIN_OCEAN", "TERRAIN_GRASS_MOUNTAIN", "TERRAIN_DESERT_MOUNTAIN", "TERRAIN_PLAINS_MOUNTAIN",
    "TERRAIN_TUNDRA_MOUNTAIN", "TERRAIN_SNOW_MOUNTAIN", "TERRAIN_LAKE",
}
IMPASSABLE_FEATURES = { "FEATURE_ICE" }

print("Constants and data structures defined.")
# print("Checking DISTRICT_PAIR_BONUS for ('Campus', 'Government Plaza'):", DISTRICT_PAIR_BONUS.get(('Campus', 'Government Plaza'), 'MISSING')) # Debug check

Constants and data structures defined.


In [192]:
# --- Helper Functions ---

def get_adjacent_tiles(x, y, all_map_coords_set):
    """ Gets coordinates of adjacent tiles based on 'odd-r' flat-top logic. """
    x = int(x)
    y = int(y)
    direct_offsets = []
    is_even_row = y % 2 == 0

    if is_even_row: # EVEN rows (y=0, 2, 4...)
        direct_offsets = [ [+1,  0], [+1, +1], [ 0, +1], [-1,  0], [ 0, -1], [+1, -1] ]
    else: # ODD rows (y=1, 3, 5...)
        direct_offsets = [ [+1,  0], [ 0, +1], [-1, +1], [-1,  0], [-1, -1], [ 0, -1] ]

    adjacent = set()
    for dx, dy in direct_offsets:
        neighbor_coords = (x + dx, y + dy)
        if neighbor_coords in all_map_coords_set:
            adjacent.add(neighbor_coords)
    return adjacent

def get_tiles_in_radius_bfs(start_coords, max_rings, all_map_coords_set):
    """ Finds all tiles within max_rings steps from start_coords using BFS. """
    start_x, start_y = int(start_coords[0]), int(start_coords[1])
    start_coords_int = (start_x, start_y)

    if start_coords_int not in all_map_coords_set:
        print(f"Warning: Start coordinates {start_coords_int} not in map data.")
        return set()
    if max_rings <= 0: return {start_coords_int}

    queue = deque([(start_coords_int, 0)])
    visited = {start_coords_int}
    reachable_tiles = {start_coords_int}

    while queue:
        current_coords, current_dist = queue.popleft()
        if current_dist < max_rings:
            neighbors = get_adjacent_tiles(current_coords[0], current_coords[1], all_map_coords_set)
            for neighbor in neighbors:
                if neighbor not in visited:
                    visited.add(neighbor)
                    reachable_tiles.add(neighbor)
                    queue.append((neighbor, current_dist + 1))
    return reachable_tiles

print("Helper functions defined.")

Helper functions defined.


In [193]:
# --- Pre-calculation Steps ---

# Set of all valid (x, y) coordinates from the DataFrame
all_map_coords = set((int(coord[0]), int(coord[1])) for coord in df[['x', 'y']].values)

# Tiles: Set of tiles within the defined radius using BFS
tiles_in_radius = get_tiles_in_radius_bfs(city_center_coords, RADIUS, all_map_coords)
print(f"Total tiles in radius {RADIUS}: {len(tiles_in_radius)}")
if not tiles_in_radius:
     print("ERROR: No tiles found within radius. Check CC coords and map data.")
     # exit()

# Mapping from (x, y) tuple to the tile's data (only for tiles in radius)
tile_data = {
    (int(row['x']), int(row['y'])): row for index, row in df.iterrows()
    if (int(row['x']), int(row['y'])) in tiles_in_radius
}

# Tiles_Workable: Subset of tiles_in_radius where districts can potentially be built
tiles_workable = set()
coast_terrain_values = FEATURES.get("Coast", ["TERRAIN_COAST"]) # Get list of coast terrain types
for coords in tiles_in_radius:
    if coords == city_center_coords: continue
    if coords not in tile_data: continue

    tile_info = tile_data[coords]
    terrain = tile_info['terrain']
    feature = tile_info.get('feature') # Use .get for features which might be null

    is_impassable_terrain = terrain in IMPASSABLE_TERRAIN
    is_impassable_feature = feature is not None and feature in IMPASSABLE_FEATURES
    is_impassable = is_impassable_terrain or is_impassable_feature

    # Allow placement on Coast tiles (for Harbor), otherwise check impassable
    if terrain in coast_terrain_values or not is_impassable:
         # Add further checks if needed (e.g., some features block certain districts)
         tiles_workable.add(coords)

print(f"Workable tiles in radius: {len(tiles_workable)}")

# Adjacency Map: Maps (x, y) -> set of adjacent (x', y') within the radius
adjacency_map = {
    coords: get_adjacent_tiles(coords[0], coords[1], all_map_coords).intersection(tiles_in_radius)
    for coords in tiles_in_radius
}

# Features on Tile Map & River Adjacency Map
features_on_tile = {}
has_river_edge = {} # Maps (x, y) -> boolean, True if tile itself has river edge data

for coords in tiles_in_radius:
    if coords not in tile_data: continue
    tile_info = tile_data[coords]
    tile_features = set() # Features relevant for providing bonuses (Mountain, Reef etc)

    terrain = tile_info['terrain']
    feature = tile_info.get('feature')
    resource = tile_info.get('resource')
    res_type = tile_info.get('resourcetype', '') # Default to empty string
    is_nw = tile_info.get('isNaturalWonder', False) # Assumes boolean flag exists
    # Add check for built Wonders if needed e.g., tile_info.get('isWonderBuilt', False)

    # Map terrain/feature names from FEATURES constant (excluding River, NW, Wonder, Resources)
    # These provide adjacency bonus
    for f_name, f_data_vals in FEATURES.items():
        # Handle bonus-providing features based on *their* terrain/feature type
         if f_name not in {"River", "NaturalWonder", "Wonder", "Strategic", "Luxury", "Floodplains", "Coast", "Lake", "Oasis"}: # Exclude types handled differently or non-bonus giving terrain itself
              if terrain in f_data_vals or (feature is not None and feature in f_data_vals):
                   tile_features.add(f_name)

    # Add special feature flags to the set for the tile itself
    if is_nw: tile_features.add("NaturalWonder")
    # if tile_info.get('isWonderBuilt', False): tile_features.add("Wonder") # Add if you track built wonders this way
    if resource:
        if res_type is not None:
            if res_type.lower() == 'strategic': tile_features.add("Strategic")
            elif res_type.lower() == 'luxury': tile_features.add("Luxury")

    features_on_tile[coords] = tile_features

    # River Edge Check (Crucial for River Bonus)
    # Assumes 'rivers' field exists and is non-empty string/list/dict if river is present
    if pd.notna(tile_info.get('rivers')) and tile_info.get('rivers') not in ["", None, [], {}]:
        has_river_edge[coords] = True
    else:
        has_river_edge[coords] = False

print(f"Tiles with river edges identified: {sum(has_river_edge.values())}")
# print({k: v for k, v in has_river_edge.items() if v}) # Optional: Print tiles with rivers

Total tiles in radius 3: 37
Workable tiles in radius: 29
Tiles with river edges identified: 0


In [194]:
# --- Pre-calculation of Boolean Maps & Adjacency ---

# isAdjacentToCC: Check adjacency for workable tiles
is_adjacent_to_cc = {
    coords: city_center_coords in adjacency_map.get(coords, set())
    for coords in tiles_workable
}

# isAdjacentToFreshwater: Check adjacency for workable tiles
# Freshwater sources: River, Lake, Oasis, Mountain (for Aqueduct)
freshwater_feature_keys = {"River", "Lake", "Oasis", "Mountain"}
is_adjacent_to_freshwater = {}
for coords in tiles_workable:
    adj_fresh = False
    for adj_coords in adjacency_map.get(coords, set()):
         # Check if adjacent tile IS a freshwater source (Lake/Oasis TILE or Mountain TILE)
         # or if the current tile has a river edge crossing to the neighbor
         if adj_coords in tile_data:
              adj_tile_info = tile_data[adj_coords]
              adj_terrain = adj_tile_info['terrain']
              adj_feature = adj_tile_info.get('feature')

              # Check Mountain, Lake, Oasis terrain/feature on adjacent tile
              if any(f_key in FEATURES and (adj_terrain in FEATURES[f_key] or (adj_feature is not None and adj_feature in FEATURES[f_key])) for f_key in {"Mountain", "Lake", "Oasis"}):
                   adj_fresh = True
                   break
              # Check if THIS tile has a river edge (more reliable than checking if neighbor IS river)
              if has_river_edge.get(coords, False):
                   adj_fresh = True
                   break # Found freshwater source (river edge)
    is_adjacent_to_freshwater[coords] = adj_fresh


# isCoast: Check terrain for workable tiles
coast_terrain_values = FEATURES.get("Coast", ["TERRAIN_COAST"])
is_coast = {
    coords: tile_data[coords]['terrain'] in coast_terrain_values
    for coords in tiles_workable if coords in tile_data
}

# isAdjacentToLand: Check adjacency for workable tiles (primarily for Harbor)
# Define land terrains (anything not Ocean, Coast, Lake - adjust if needed)
land_terrains = { t for t in df['terrain'].unique() if t not in {"TERRAIN_OCEAN", "TERRAIN_COAST", "TERRAIN_LAKE"} }
is_adjacent_to_land = {}
for coords in tiles_workable:
    # Only relevant if the tile itself is Coast
    if is_coast.get(coords, False):
        adj_land = False
        for adj_coords in adjacency_map.get(coords, set()):
             if adj_coords in tile_data and tile_data[adj_coords]['terrain'] in land_terrains:
                  adj_land = True
                  break
        is_adjacent_to_land[coords] = adj_land
    else:
        is_adjacent_to_land[coords] = False # Not relevant if the tile itself isn't Coast

# isFloodplains: Check feature for workable tiles (for Dam)
floodplains_feature_values = FEATURES.get("Floodplains", [])
is_floodplains = {
     coords: tile_data[coords].get('feature') in floodplains_feature_values
     for coords in tiles_workable if coords in tile_data
}

print("Pre-calculation of boolean maps finished.")

Pre-calculation of boolean maps finished.


In [195]:
# --- Initialize Gurobi Model ---
model = Model("Civ6_District_Optimization")
model.setParam('OutputFlag', 1) # Set to 1 for solver output, 0 to suppress
print("Gurobi model initialized.")

Set parameter OutputFlag to value 1
Gurobi model initialized.


In [196]:
# --- Define Main Decision Variables: Build(x, y, d) ---
build_vars = {}
print("Creating main 'Build' variables...")
for coords in tiles_workable:
    for d in ALL_DISTRICTS:
        var_name = f"Build[{coords[0]},{coords[1]},{d}]"
        build_vars[coords, d] = model.addVar(vtype=GRB.BINARY, name=var_name)

# Integrate new variables immediately (good practice)
model.update()
print(f"Gurobi 'Build' variables defined ({len(build_vars)} variables).")

Creating main 'Build' variables...
Gurobi 'Build' variables defined (348 variables).


In [197]:
# --- Define Auxiliary Variables for District Adjacency ---
adj_district_vars = {} # Format: adj_district_vars[coords_d1, d1, d2] = variable A

print("Defining auxiliary adjacency variables...")
# Create variables only for pairs defined in DISTRICT_PAIR_BONUS where d1 receives bonus from d2
for (d1, d2), bonus_value in DISTRICT_PAIR_BONUS.items():
    if bonus_value > 0:
        # Check if d2 is a district type or a feature like 'Wonder'
        is_d2_district = d2 in ALL_DISTRICTS
        is_d2_feature = d2 in FEATURES # e.g., 'Wonder' if handled as feature

        for coords_d1 in tiles_workable:
            # Ensure the primary district (d1) variable exists for this tile
            if (coords_d1, d1) in build_vars:
                 # Create the aux variable: AdjDist_x_y_d1_from_d2 = 1 if d1 at (x,y) gets bonus from adjacent d2
                 var_name = f"AdjDist_{coords_d1[0]}_{coords_d1[1]}_{d1}_from_{d2}"
                 # Use tuple key: (coords_tuple, d1_string, d2_string)
                 current_key = (coords_d1, d1, d2) # Define the key explicitly
                 adj_district_vars[current_key] = model.addVar(vtype=GRB.BINARY, name=var_name)

                 # --- Add specific debug print for variable creation ---
                 if coords_d1 == (51, 16) and d1 == 'Campus' and d2 == 'Government Plaza':
                      print(f"\n*** DEBUG: SUCCESSFULLY CREATED aux var for key {current_key} with name {var_name} ***\n")
                 # --- End debug print ---


# Integrate new variables
model.update()
print(f"Defined {len(adj_district_vars)} auxiliary adjacency variables.")
# Check if the specific key exists AFTER the loop
check_key = ((51, 16), 'Campus', 'Government Plaza')
if check_key in adj_district_vars:
    print(f"DEBUG: Key {check_key} EXISTS in adj_district_vars after creation loop.")
else:
    print(f"*** CRITICAL WARNING: Key {check_key} DOES NOT EXIST in adj_district_vars after creation loop! ***")

Defining auxiliary adjacency variables...

*** DEBUG: SUCCESSFULLY CREATED aux var for key ((51, 16), 'Campus', 'Government Plaza') with name AdjDist_51_16_Campus_from_Government Plaza ***

Defined 348 auxiliary adjacency variables.
DEBUG: Key ((51, 16), 'Campus', 'Government Plaza') EXISTS in adj_district_vars after creation loop.


In [ ]:
# --- Add Linking Constraints for Auxiliary Adjacency Variables ---
# Links A = adj_district_vars[coords_d1, d1, d2]
# To B1 = build_vars[coords_d1, d1]
# And S2 = Sum over adjacent tiles (adj_coords) of Build(adj_coords, d2) or HasFeature(adj_coords, d2)

print("Adding linking constraints for adjacency variables...")
constraints_added_count = 0

for (coords_d1, d1, d2), A in adj_district_vars.items():
    # Get the B1 variable (Build(d1) at coords_d1)
    if (coords_d1, d1) not in build_vars: continue # Should not happen based on creation logic, but safe check
    B1 = build_vars[coords_d1, d1]

    # Calculate S2 = Sum of adjacent triggers (d2)
    adjacent_tiles = adjacency_map.get(coords_d1, set())
    sum_expr_S2 = None # Initialize S2 expression

    # Determine if d2 is a District or a Feature triggering the bonus
    if d2 in ALL_DISTRICTS:
         # S2 is the sum of Build(adj_coords, d2) for adjacent workable tiles
         # Use a list comprehension inside quicksum for clarity if needed
         s2_terms = [build_vars.get((adj_coords, d2), 0)
                     for adj_coords in adjacent_tiles
                     if adj_coords in tiles_workable and (adj_coords, d2) in build_vars] # Ensure var exists
         sum_expr_S2 = quicksum(s2_terms)

    elif d2 in FEATURES: # Check if d2 is a key in FEATURES dict (e.g., "Wonder")
         # S2 is the sum of indicator constants (1 if feature present, 0 otherwise)
         sum_expr_S2 = quicksum(1
                                for adj_coords in adjacent_tiles
                                if adj_coords in features_on_tile and d2 in features_on_tile[adj_coords])
    else:
         # Skip if d2 is not a district or known feature key
         continue

    # --- DEBUG PRINT for S2 expression (Now Included) ---
    if d1 == 'Campus' and d2 == 'Government Plaza' and coords_d1 == (51, 16): # Check for specific Campus location
         print(f"\n*** DEBUG constraint S2 check for Campus at {coords_d1} from GP: ***")
         try:
              # Check if S2 is a valid Gurobi expression type
              if isinstance(sum_expr_S2, (int, float, type(None))): # Check if it resolved to a constant or None
                   print(f"  S2 Expression evaluated to constant/None: {sum_expr_S2}")
              else: # Assume it's a Gurobi LinExpr
                   print(f"  S2 Expression (Gurobi): {sum_expr_S2}")

              # Check components manually to see which vars S2 depends on
              terms = []
              s2_term_vars_found = False
              for adj_coords_debug in adjacency_map.get(coords_d1, set()):
                    if adj_coords_debug in tiles_workable:
                         key_debug = (adj_coords_debug, 'Government Plaza')
                         if key_debug in build_vars:
                              terms.append(build_vars[key_debug].VarName)
                              s2_term_vars_found = True
              print(f"  S2 depends on GP Build Vars on workable adjacent tiles: {terms if s2_term_vars_found else '*** NONE FOUND - THIS IS LIKELY THE PROBLEM ***'}")
         except Exception as e:
              print(f"  Error printing S2 expression/components: {e}")
         print("*** End S2 Check ***\n")
    # --- End S2 Debug Print ---


    # --- Constraints linking A to B1 and S2: A = 1 iff B1=1 and S2>=1 ---
    constr_name_base = f"LinkAdj_{coords_d1[0]}_{coords_d1[1]}_{d1}_{d2}"

    # Check if sum_expr_S2 is valid before adding constraints using it
    if sum_expr_S2 is not None:
         # 1. A must be 0 or 1
         model.addConstr(A <= 1)
          
         # 2. A can only be 1 if B1 is 1
         model.addConstr(A <= B1)
          
         # 3. Create indicator for adjacency
         has_adj = model.addVar(vtype=GRB.BINARY, name=f"{constr_name_base}_HasAdj")
         model.addConstr(sum_expr_S2 <= MAX_ADJACENT_TILES * has_adj)
         model.addConstr(has_adj * 0.1 <= sum_expr_S2)
          
         # 4. A can only be 1 if there's adjacency
         model.addConstr(A <= has_adj)
          
         # 5. A must be 1 if both B1 and has_adj are 1
         model.addConstr(A >= B1 + has_adj - 1)

     # else:
     #      print(f"Warning: sum_expr_S2 was None for {constr_name_base}. Constraints not added") # Optional


print(f"Added {constraints_added_count} linking constraints for adjacency.")

Adding linking constraints for adjacency variables...

*** DEBUG constraint S2 check for Campus at (51, 16) from GP: ***
  S2 Expression (Gurobi): Build[51,15,Government Plaza] + Build[52,17,Government Plaza] + Build[50,16,Government Plaza] + Build[52,16,Government Plaza] + Build[51,17,Government Plaza]
  S2 depends on GP Build Vars on workable adjacent tiles: ['Build[51,15,Government Plaza]', 'Build[52,17,Government Plaza]', 'Build[50,16,Government Plaza]', 'Build[52,16,Government Plaza]', 'Build[51,17,Government Plaza]']
*** End S2 Check ***

Added 1044 linking constraints for adjacency.


In [199]:
# --- Define Objective Function ---

print("Calculating objective function coefficients...")

# Calculate base coefficients from Terrain, Features (non-district), CC, River
obj_coeffs = {}
for coords in tiles_workable:
    for d in ALL_DISTRICTS:
        # Sum terrain/feature bonuses from adjacent tiles (non-River)
        terrain_feature_bonus = 0
        for adj_coords in adjacency_map.get(coords, set()):
             if adj_coords in features_on_tile:
                  for f in features_on_tile.get(adj_coords, set()):
                       # Look up bonus value, ensuring f isn't the River key used for edge logic
                       if f != RIVER_FEATURE_KEY:
                           terrain_feature_bonus += TERRAIN_BONUS_VALUE.get((d, f), 0)

        # Add City Center bonus if adjacent
        cc_bonus = CC_BONUS_VALUE.get(d, 0) if is_adjacent_to_cc.get(coords, False) else 0

        # Add River Adjacency Bonus if *this tile* has a river edge
        river_bonus = 0
        if has_river_edge.get(coords, False):
            river_bonus = TERRAIN_BONUS_VALUE.get((d, RIVER_FEATURE_KEY), 0)

        obj_coeffs[coords, d] = terrain_feature_bonus + cc_bonus + river_bonus

print("Calculating base objective expression...")
# Base objective part from terrain, CC, river bonuses
base_objective_expr = quicksum(
    build_vars[key] * obj_coeffs.get(key, 0)
    for key in build_vars.keys() # Iterate through all defined build variables
)

print("Calculating district adjacency bonus contribution...")
# District adjacency bonus part using the auxiliary variables
district_adj_bonus_expr_terms = []
processed_obj_keys = set() # Keep track of keys processed in this loop

for key, aux_var in adj_district_vars.items():
    # Ensure key is the expected tuple format (coords_tuple, d1_string, d2_string)
    if not (isinstance(key, tuple) and len(key) == 3):
         print(f"WARNING: Unexpected key format in adj_district_vars: {key}")
         continue
    coords_d1, d1, d2 = key # Unpack the key
    processed_obj_keys.add(key) # Mark this key as processed

    bonus = DISTRICT_PAIR_BONUS.get((d1, d2), 0)
    if bonus > 0:
        term = aux_var * bonus
        district_adj_bonus_expr_terms.append(term)

        # --- Add specific debug print for GP bonus ---
        if coords_d1 == (51, 16) and d1 == 'Campus' and d2 == 'Government Plaza':
            print(f"\n*** DEBUG: FOUND AND ADDING objective term for {aux_var.VarName} * {bonus} (GP->Campus Bonus) ***\n")
        # --- End debug print ---

# Check if the specific key we care about was missed in the loop
check_key_obj = ((51, 16), 'Campus', 'Government Plaza')
if check_key_obj in adj_district_vars and check_key_obj not in processed_obj_keys:
     print(f"*** CRITICAL WARNING: Key {check_key_obj} exists in adj_district_vars but WAS NOT processed in objective loop! ***")


# District base value contribution 
district_base_value_expr = quicksum(
    build_vars[key] * DISTRICT_BASE_VALUE.get(key[1], 0.0)  # key[1] is the district type
    for key in build_vars.keys()
)

district_adj_bonus_expr = quicksum(district_adj_bonus_expr_terms)

# --- Final Objective Check & Setting ---
print("\n--- Final Objective Check ---")
print(f"Base Objective Expr Type: {type(base_objective_expr)}")
print(f"District Adj Bonus Expr Type: {type(district_adj_bonus_expr)}")

# Combine objective parts
final_obj = base_objective_expr + district_adj_bonus_expr + district_base_value_expr
print(f"Final Objective Expr Type: {type(final_obj)}")
print("Setting final objective in Gurobi...")
model.setObjective(final_obj, GRB.MAXIMIZE)
print("--- End Final Objective Check ---")

print("Gurobi objective function defined.")

Calculating objective function coefficients...
Calculating base objective expression...
Calculating district adjacency bonus contribution...

*** DEBUG: FOUND AND ADDING objective term for AdjDist_51_16_Campus_from_Government Plaza * 1 (GP->Campus Bonus) ***


--- Final Objective Check ---
Base Objective Expr Type: <class 'gurobipy._core.LinExpr'>
District Adj Bonus Expr Type: <class 'gurobipy._core.LinExpr'>
Final Objective Expr Type: <class 'gurobipy._core.LinExpr'>
Setting final objective in Gurobi...
--- End Final Objective Check ---
Gurobi objective function defined.


In [200]:
# --- Add Core Constraints ---
print("Defining core constraints...")

# 1. One District Per Workable Tile
for coords in tiles_workable:
    model.addConstr(quicksum(build_vars[coords, d] for d in ALL_DISTRICTS if (coords, d) in build_vars) <= 1,
                    name=f"OneDistPerTile_{coords[0]}_{coords[1]}")

# 2. Unique District Limit
# Ensure DISTRICTS_UNIQUE has correct, consistent names
for d in DISTRICTS_UNIQUE:
    # Check if the district 'd' is actually in the list of districts we are considering
    if d in ALL_DISTRICTS:
         model.addConstr(quicksum(build_vars[coords, d] for coords in tiles_workable if (coords, d) in build_vars) <= 1,
                         name=f"UniqueDistrict_{d}")
    # else:
    #      print(f"Warning: District '{d}' in DISTRICTS_UNIQUE but not in ALL_DISTRICTS. Skipping uniqueness constraint.")


# 3. Mutual Exclusivity
# Ensure district names in DISTRICTS_MUTUALLY_EXCLUSIVE are correct
for i, me_set in enumerate(DISTRICTS_MUTUALLY_EXCLUSIVE):
    valid_me_districts = [d for d in me_set if d in ALL_DISTRICTS] # Filter for districts actually in model
    if valid_me_districts:
        expr = quicksum(build_vars[coords, d]
                        for coords in tiles_workable
                        for d in valid_me_districts
                        if (coords, d) in build_vars) # Check key exists
        model.addConstr(expr <= 1, name=f"MutExcl_{i}")

print("Core constraints defined.")

Defining core constraints...
Core constraints defined.


In [201]:
# --- Add Specific District Placement Constraints ---
print("Defining placement constraints...")

# 4. Aqueduct Placement Requirements (Adj to CC & Freshwater)
AQUEDUCT = "Aqueduct"
if AQUEDUCT in ALL_DISTRICTS:
    for coords in tiles_workable:
        key = (coords, AQUEDUCT)
        if key in build_vars: # Check if variable exists
            # Force 0 if not adjacent to CC
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_cc.get(coords, False) else 0),
                            name=f"AqueductAdjCC_{coords[0]}_{coords[1]}")
            # Force 0 if not adjacent to Freshwater
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_freshwater.get(coords, False) else 0),
                            name=f"AqueductFreshwater_{coords[0]}_{coords[1]}")

# 5. Harbor Placement Requirements (Coast & Adj to Land)
HARBOR = "Harbor"
if HARBOR in ALL_DISTRICTS:
    for coords in tiles_workable:
        key = (coords, HARBOR)
        if key in build_vars:
            # Force 0 if not Coast
            model.addConstr(build_vars[key] <= (1 if is_coast.get(coords, False) else 0),
                            name=f"HarborIsCoast_{coords[0]}_{coords[1]}")
            # Force 0 if not adjacent to Land
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_land.get(coords, False) else 0),
                            name=f"HarborAdjLand_{coords[0]}_{coords[1]}")

# 6. Dam Placement Requirements (On Floodplains & On River)
DAM = "Dam"
if DAM in ALL_DISTRICTS:
    for coords in tiles_workable:
         key = (coords, DAM)
         if key in build_vars:
              # Force 0 if not on Floodplains tile
              model.addConstr(build_vars[key] <= (1 if is_floodplains.get(coords, False) else 0),
                              name=f"DamIsFloodplains_{coords[0]}_{coords[1]}")
              # Force 0 if tile does not have a river edge
              model.addConstr(build_vars[key] <= (1 if has_river_edge.get(coords, False) else 0),
                              name=f"DamIsRiver_{coords[0]}_{coords[1]}")

# 7. Canal Placement Requirements (Example - needs specific logic based on spanning water/cities/coast)
CANAL = "Canal"
if CANAL in ALL_DISTRICTS:
     # Add specific constraints for Canal placement if needed.
     # E.g., must be on flat land, connect two water bodies or city center + water, max length etc.
     # This requires more complex geometric data not currently pre-calculated.
     pass # Placeholder

# # Add constraints for other districts like Government Plaza if they have special rules
# # (e.g., cannot be adjacent to CC - though standard rules allow this)
# GOVERNMENT_PLAZA = "Government Plaza"
# if GOVERNMENT_PLAZA in ALL_DISTRICTS:
#     for coords in tiles_workable:
#         key = (coords, GOVERNMENT_PLAZA)
#         if key in build_vars:
#             # Example: Force 0 if adjacent to CC
#             # model.addConstr(build_vars[key] <= (0 if is_adjacent_to_cc.get(coords, False) else 1),
#             #                 name=f"GovPlazaNotAdjCC_{coords[0]}_{coords[1]}")
#             pass # No special constraints added by default

print("Placement constraints defined.")

Defining placement constraints...
Placement constraints defined.


In [202]:
# # Ensure this block is active BEFORE Cell 14 (model.optimize())
# print("\n--- DEBUG: Adding temporary constraint to force GP placement ---")
# forced_gp_loc = (55, 23)
# gp_build_key = (forced_gp_loc, 'Government Plaza')
# if gp_build_key in build_vars:
#     print(f"Forcing build_vars['{gp_build_key}'] == 1")
#     force_gp_constr = model.addConstr(build_vars[gp_build_key] == 1, name="ForceGP_Debug")
#     model.update()
#     print(f"Constraint '{force_gp_constr.ConstrName}' added.")
# else:
#     print(f"WARNING: Cannot add force constraint, build variable {gp_build_key} missing!")
# print("--- End Temporary Constraint ---\n")

In [203]:
# --- Sanity Checks (Run Before Solving) ---

print("\n--- Running Sanity Checks ---")

# 1. Harbor Placement Check
# ... (keep existing Harbor check code) ...
print("Checking potential Harbor tiles...")
potential_harbor_tiles = [
    coords for coords in tiles_workable
    if is_coast.get(coords, False) and is_adjacent_to_land.get(coords, False)
]
if not potential_harbor_tiles:
    print("WARNING: No workable tiles meet Harbor placement requirements (Coast & Adj Land)!")
else:
    print(f"  Found {len(potential_harbor_tiles)} potential Harbor tiles: {potential_harbor_tiles}")


# 2. Government Plaza Variable Check
# ... (keep existing GP check code) ...
print("Checking Government Plaza definitions...")
gp_in_all = 'Government Plaza' in ALL_DISTRICTS
gp_in_unique = 'Government Plaza' in DISTRICTS_UNIQUE
print(f"  GP in ALL_DISTRICTS: {gp_in_all}")
print(f"  GP in DISTRICTS_UNIQUE: {gp_in_unique}")
gp_vars_exist = any(d == 'Government Plaza' for _, d in build_vars.keys())
print(f"  Build variables created for Government Plaza: {gp_vars_exist}")
if not gp_vars_exist and gp_in_all:
     print("  WARNING: GP is in ALL_DISTRICTS but no build variables found!")


# 3. Harbor Variable Check
# ... (keep existing Harbor check code) ...
print("Checking Harbor definitions...")
harbor_in_all = 'Harbor' in ALL_DISTRICTS
harbor_in_unique = 'Harbor' in DISTRICTS_UNIQUE
print(f"  Harbor in ALL_DISTRICTS: {harbor_in_all}")
print(f"  Harbor in DISTRICTS_UNIQUE: {harbor_in_unique}")
harbor_vars_exist = any(d == 'Harbor' for _, d in build_vars.keys())
print(f"  Build variables created for Harbor: {harbor_vars_exist}")
if harbor_vars_exist:
    harbor_var_count = sum(1 for _, d in build_vars.keys() if d == 'Harbor')
    print(f"    Number of Harbor build variables: {harbor_var_count}")
elif harbor_in_all:
    print("  WARNING: Harbor is in ALL_DISTRICTS but no build variables found!")


# 4. GP Base Coefficient Check (Added/Ensured Present)
print("Checking GP Base Coefficients on Tiles Adjacent to Campus (51, 16)...")
campus_loc = (51, 16)
if campus_loc in adjacency_map:
    workable_adj_to_campus = adjacency_map[campus_loc].intersection(tiles_workable)
    if not workable_adj_to_campus:
         print("  Cannot check GP coeffs: No workable tiles adjacent to campus found.")
    else:
         found_negative_gp_coeff = False
         for adj_coords in workable_adj_to_campus:
              gp_key = (adj_coords, 'Government Plaza')
              # Check obj_coeffs dictionary calculated in Cell 10
              coeff = obj_coeffs.get(gp_key, 0) # Default to 0 if missing
              print(f"  obj_coeffs for GP at {adj_coords}: {coeff}")
              if coeff < 0:
                   print(f"  ---> WARNING: Negative base coefficient found for GP at {adj_coords}!")
                   found_negative_gp_coeff = True
         if not found_negative_gp_coeff:
              print("  (All checked GP base coefficients are >= 0)")
else:
    print(f"  Cannot check GP coeffs: Campus location {campus_loc} not found in adjacency_map.")


print("--- End Sanity Checks ---\n")


--- Running Sanity Checks ---
Checking potential Harbor tiles...
  Found 7 potential Harbor tiles: [(53, 17), (53, 19), (51, 15), (52, 17), (53, 18), (54, 20), (52, 16)]
Checking Government Plaza definitions...
  GP in ALL_DISTRICTS: True
  GP in DISTRICTS_UNIQUE: True
  Build variables created for Government Plaza: True
Checking Harbor definitions...
  Harbor in ALL_DISTRICTS: True
  Harbor in DISTRICTS_UNIQUE: True
  Build variables created for Harbor: True
    Number of Harbor build variables: 29
Checking GP Base Coefficients on Tiles Adjacent to Campus (51, 16)...
  obj_coeffs for GP at (51, 15): 0
  obj_coeffs for GP at (52, 17): 0
  obj_coeffs for GP at (50, 16): 0
  obj_coeffs for GP at (52, 16): 0
  obj_coeffs for GP at (51, 17): 0
  (All checked GP base coefficients are >= 0)
--- End Sanity Checks ---



In [204]:
# --- Solve the Optimization Problem ---
print("Solving the optimization problem using Gurobi...")
# Optional: Write model to file for debugging
# model.write("civ6_model_debug.lp")
# print("Model written to civ6_model_debug.lp")

model.optimize()

Solving the optimization problem using Gurobi...
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3000 rows, 1044 columns and 11308 nonzeros
Model fingerprint: 0xdedd26c0
Variable types: 0 continuous, 1044 integer (1044 binary)
Coefficient statistics:
  Matrix range     [1e-01, 6e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 2966 rows and 899 columns
Presolve time: 0.01s
Presolved: 34 rows, 145 columns, 290 nonzeros
Variable types: 0 continuous, 145 integer (145 binary)

Root relaxation: objective 1.350000e+01, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [205]:
# --- Print the Optimization Results ---
print("\n--- Gurobi Optimization Results ---")

if model.Status == GRB.OPTIMAL:
    print(f"Status: Optimal")
    print(f"Optimal Total Adjacency Score = {model.ObjVal:.2f}")
    print("\nOptimal District Placements:")
    placed_districts = 0
    solution_placements = []
    # Iterate through build_vars dictionary
    for key, var in build_vars.items():
         # Use a tolerance for checking binary variable solution
         if var.X > 0.5:
              coords, district_name = key
              solution_placements.append((district_name, coords[0], coords[1]))
              placed_districts += 1

    # Sort placements for readability (by district name, then coords)
    solution_placements.sort()
    for placement in solution_placements:
        print(f"  Build {placement[0]} at ({placement[1]}, {placement[2]})")

    if placed_districts == 0:
        print("  No districts were placed in the optimal solution.")

# --- Modify this block within Cell 15 ---

# --- Modify this block within Cell 15 AGAIN ---

elif model.Status == GRB.INFEASIBLE:
    print("Status: Infeasible - The problem has no solution.")
    print("Check constraints, data (especially workable tiles), and bonus values.")
    # Compute and print IIS (Irreducible Inconsistent Subsystem) to help debug
    print("\nComputing IIS to identify conflicting constraints...")
    try:
        model.computeIIS() # Compute the IIS first
        model.update() # Ensure model state is updated after IIS computation

        # --- Get constraints using indices from IIS results ---
        print("Conflicting Constraints:")
        if hasattr(model, 'IISConstr'):
            # Get the list of all linear constraints in the model
            all_constrs = model.getConstrs()
            # IISConstr likely contains indices into this list
            iis_constr_indices = model.IISConstr
            if not iis_constr_indices:
                 print("  No conflicting constraints found in IISConstr list.")
            else:
                 for index in iis_constr_indices:
                      try:
                           # Use the index to get the actual constraint object
                           constraint = all_constrs[index]
                           print(f"  {constraint.ConstrName}")
                      except IndexError:
                           print(f"  Error: Invalid constraint index {index} found in IISConstr.")
                      except AttributeError:
                           print(f"  Error: Object at index {index} has no ConstrName.")
        else:
             print("  Could not retrieve IIS Constraints (model.IISConstr attribute missing).")


        # --- Get variable bounds using indices from IIS results ---
        print("Conflicting Variable Bounds:")
        lb_conflict = False
        ub_conflict = False
        if hasattr(model, 'IISLB') or hasattr(model, 'IISUB'):
             # Get the list of all variables in the model
             all_vars = model.getVars()

             if hasattr(model, 'IISLB'):
                  iis_lb_indices = model.IISLB
                  if iis_lb_indices:
                       lb_conflict = True
                       for index in iis_lb_indices:
                            try:
                                 variable = all_vars[index]
                                 print(f"  {variable.VarName} (Lower Bound)")
                            except IndexError:
                                 print(f"  Error: Invalid variable index {index} found in IISLB.")
                            except AttributeError:
                                 print(f"  Error: Object at index {index} has no VarName.")
                  # else: print("  No conflicting lower bounds found.") # Optional message

             if hasattr(model, 'IISUB'):
                  iis_ub_indices = model.IISUB
                  if iis_ub_indices:
                       ub_conflict = True
                       for index in iis_ub_indices:
                            try:
                                 variable = all_vars[index]
                                 print(f"  {variable.VarName} (Upper Bound)")
                            except IndexError:
                                 print(f"  Error: Invalid variable index {index} found in IISUB.")
                            except AttributeError:
                                 print(f"  Error: Object at index {index} has no VarName.")
                  # else: print("  No conflicting upper bounds found.") # Optional message

        if not lb_conflict and not ub_conflict:
             print("  No conflicting variable bounds found in IIS lists.")


        # Optional: Write IIS to a file for detailed analysis
        # model.write("civ6_infeasible.ilp")
        # print("IIS written to civ6_infeasible.ilp")

    except GurobiError as iis_error:
        print(f"Could not compute IIS or get constraints/variables: {iis_error}")
    except AttributeError as attr_error:
         # Catch potential lingering issues or version differences
         print(f"Attribute Error during IIS processing (check Gurobi version/API): {attr_error}")
    except Exception as e:
         print(f"An error occurred during IIS computation: {e}")

# --- End modification ---


elif model.Status == GRB.UNBOUNDED:
    print("Status: Unbounded - The objective can be increased indefinitely.")
    print("This usually indicates missing constraints or an error in the objective function/coefficients.")
else:
    # Consult Gurobi documentation for other status codes
    print(f"Status Code: {model.Status} (Not Optimal/Infeasible/Unbounded)")
    print("Solver did not find an optimal solution. Check Gurobi logs or status code meaning.")

print("--- End Results ---")


--- Gurobi Optimization Results ---
Status: Optimal
Optimal Total Adjacency Score = 13.50

Optimal District Placements:
  Build Campus at (53, 18)
  Build Commercial Hub at (51, 20)
  Build Holy Site at (53, 19)
  Build Industrial Zone at (53, 21)
  Build Theater Square at (52, 21)
--- End Results ---


In [206]:
# --- Debugging Checks (Run After Printing Results) ---

# print contribution of district base value to objective
if model.Status == GRB.OPTIMAL:
    district_base_value = sum(DISTRICT_BASE_VALUE.get(key[1], 0) * var.X 
                             for key, var in build_vars.items() if var.X > 0.5)
    print(f"District Base Value Contribution: {district_base_value:.2f}")

# Add after solving to check why GP isn't placed
if model.Status == GRB.OPTIMAL:
    print("District-to-district adjacency contributions:")
    for key, var in adj_district_vars.items():
        if var.X > 0.5:
            coords, d1, d2 = key
            bonus = DISTRICT_PAIR_BONUS.get((d1, d2), 0)
            print(f"  {d1} at {coords} gets +{bonus} from adjacent {d2}")
        else:
            # print(f"  {key} not placed in solution (var.X = {var.X:.2f})")
            pass

# Find optimal campus location if one was placed
optimal_campus_loc = None
if model.Status == GRB.OPTIMAL:
    for key, var in build_vars.items():
        if var.X > 0.5 and key[1] == 'Campus':
            optimal_campus_loc = key[0]
            break # Assume only one campus

if optimal_campus_loc:
    print(f"\n--- Debugging GP Adjacency to Optimal Campus at {optimal_campus_loc} ---")

    # Check if the optimal location is in the adjacency map
    if optimal_campus_loc not in adjacency_map:
        print(f"ERROR: Optimal Campus location {optimal_campus_loc} not found in adjacency_map keys!")
    else:
        adj_to_campus = adjacency_map[optimal_campus_loc]
        print(f"Tiles adjacent to Campus {optimal_campus_loc}: {adj_to_campus}")

        # Find which of these adjacent tiles are actually workable
        workable_adj_to_campus = adj_to_campus.intersection(tiles_workable)
        print(f"WORKABLE tiles adjacent to Campus: {workable_adj_to_campus}")

        if not workable_adj_to_campus:
            print("\nCONCLUSION: There are NO workable tiles adjacent to the optimal Campus location.")
            print("The Government Plaza cannot be placed adjacent to the Campus, so its bonus cannot be achieved.")
        else:
            print("\nINFO: There ARE workable tiles adjacent to the Campus where GP could potentially be placed.")
            # Further check if GP build vars exist for these specific tiles
            gp_build_vars_on_adj = []
            for adj_coords in workable_adj_to_campus:
                 gp_key = (adj_coords, 'Government Plaza')
                 if gp_key in build_vars:
                      gp_build_vars_on_adj.append(build_vars[gp_key].VarName)
                 else:
                      print(f"  WARNING: Build variable for GP not found for workable adjacent tile {adj_coords}!")
            if gp_build_vars_on_adj:
                 print(f"  GP build variables exist for adjacent workable tiles: {len(gp_build_vars_on_adj)} found.")

    # Verify the specific auxiliary variable exists and the bonus value is read correctly
    adj_var_key = (optimal_campus_loc, 'Campus', 'Government Plaza')
    if adj_var_key in adj_district_vars:
         print(f"Auxiliary variable for Campus <- GP bonus exists: {adj_district_vars[adj_var_key].VarName}")
         # Use the *actual* dictionary value used in the objective calculation
         bonus_val = DISTRICT_PAIR_BONUS.get(('Campus', 'Government Plaza'), -999) # Use -999 to show if key missing
         print(f"Bonus value ('Campus', 'Government Plaza') read from dict: {bonus_val}")
         # Check if the variable was active in the solution (if GP bonus was expected)
         if model.Status == GRB.OPTIMAL:
             print(f"  Value of Aux Var in solution: {adj_district_vars[adj_var_key].X:.0f}")

    else:
         print(f"ERROR: Auxiliary variable {adj_var_key} DOES NOT exist. Check its creation loop.")

    print("--- End GP Debug ---")
elif model.Status == GRB.OPTIMAL:
    print("\n--- GP Debug Skipped: No Campus placed in optimal solution. ---")

District Base Value Contribution: 10.00
District-to-district adjacency contributions:

--- Debugging GP Adjacency to Optimal Campus at (53, 18) ---
Tiles adjacent to Campus (53, 18): {(54, 19), (53, 17), (54, 18), (53, 19), (52, 18), (54, 17)}
WORKABLE tiles adjacent to Campus: {(54, 18), (53, 17), (53, 19), (54, 17)}

INFO: There ARE workable tiles adjacent to the Campus where GP could potentially be placed.
  GP build variables exist for adjacent workable tiles: 4 found.
Auxiliary variable for Campus <- GP bonus exists: AdjDist_53_18_Campus_from_Government Plaza
Bonus value ('Campus', 'Government Plaza') read from dict: 1
  Value of Aux Var in solution: 0
--- End GP Debug ---
